In [388]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from pandas_profiling import ProfileReport
from sklearn.ensemble import RandomForestClassifier
from scipy.special import boxcox1p
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, classification_report 
from sklearn.metrics import plot_confusion_matrix, precision_recall_curve, plot_precision_recall_curve, plot_roc_curve

import lightgbm as lgb
warnings.filterwarnings('ignore')
%matplotlib inline


In [389]:
X = pd.read_csv("data/train.csv", index_col='PassengerId')
X_test = pd.read_csv("data/test.csv", index_col='PassengerId')
X.drop(['Cabin'], axis=1, inplace=True)
X_test.drop(['Cabin'], axis=1, inplace=True)
X.drop(['Ticket'], axis=1, inplace=True)
X_test.drop(['Ticket'], axis=1, inplace=True)
X.Embarked = X.Embarked.fillna('S')
X_test.Embarked = X_test.Embarked.fillna('S')

def cleanage(X):
    X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch > 0)] = X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch == 0)] = X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 3) & (X.Sex == 'male') & (X.Parch == 0)].mean())

    X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch > 0)] = X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch == 0)] = X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 3) & (X.Sex == 'female') & (X.Parch == 0)].mean())

    X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch > 0)] = X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch == 0)] = X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 2) & (X.Sex == 'male') & (X.Parch == 0)].mean())

    X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch > 0)] = X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch == 0)] = X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 2) & (X.Sex == 'female') & (X.Parch == 0)].mean())

    X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch > 0)] = X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch == 0)] = X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 1) & (X.Sex == 'male') & (X.Parch == 0)].mean())

    X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch > 0)] = X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch > 0)].fillna(X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch > 0)].mean())
    X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch == 0)] = X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch == 0)].fillna(X['Age'][(X.Pclass == 1) & (X.Sex == 'female') & (X.Parch == 0)].mean())
        #unskew Fare
    lam = 0.1
    X['Fare'] = boxcox1p(X['Fare'], lam)
    
    #ord Pclass
    X['Pclass'] = X['Pclass'].astype(str)
    X['hasparchild'] = X.Parch > 0
    X['hassibspouse'] = X.SibSp > 0
    X.drop(['Parch'], axis=1, inplace=True)
    X.drop(['SibSp'], axis=1, inplace=True)
    
    X['Agegroup'] = ""
    X['Agegroup'][X.Age < 10] = "0 to 9"
    X['Agegroup'][(X.Age >= 10) & (X.Age < 30)] = "10 to 29"
    X['Agegroup'][(X.Age >= 30) & (X.Age < 40)] = "30 to 39"
    X['Agegroup'][(X.Age >= 40) & (X.Age < 50)] = "40 to 49"
    X['Agegroup'][(X.Age >= 50) & (X.Age < 60)] = "50 to 59"
    X['Agegroup'][(X.Age >= 60)] = ">= 60"
    X.drop(['Age'], axis=1, inplace=True)
    return X

cleanage(X)
cleanage(X_test)




class_categories = ["3","2","1"] 
sex_categories = ['male', 'female']
embarked_categories = ['S', 'C', 'Q']
age_categories = ['0 to 9'
                  , '10 to 29'
                  , '30 to 39'
                  , '40 to 49'
                  , '50 to 59'
                  , '>= 60'
                 ]


encoder = OrdinalEncoder(categories=[class_categories, sex_categories, embarked_categories, age_categories])
X[['Pclass','Sex', 'Embarked','Agegroup']] = encoder.fit_transform(X[['Pclass','Sex', 'Embarked','Agegroup']])
X_test[['Pclass','Sex', 'Embarked','Agegroup']] = encoder.transform(X_test[['Pclass','Sex', 'Embarked','Agegroup']])

X_test.Fare = X_test.Fare.fillna(X_test.Fare[(X_test.Pclass==0.0) & (X_test.Sex == 0.0) & (X_test.Embarked == 0.0) & (X_test.hasparchild == False) & (X_test.hassibspouse == False)].mean())


y = X.pop('Survived')



In [390]:
X.drop(['Name'], axis=1, inplace=True)
X_test.drop(['Name'], axis=1, inplace=True)


In [391]:

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)    

randfor = RandomForestClassifier(n_estimators=100)

randfor.fit(X_train,y_train)

preds = randfor.predict(X_valid)

acc_random_forest = round(randfor.score(X_train, y_train) * 100, 2)
acc_random_forest

93.82

In [392]:
LGBM = lgb.LGBMRegressor()

LGBM.fit(X_train,y_train)
lgbmpreds = LGBM.predict(X_valid)

acc_lgbm = round(LGBM.score(X_train, y_train) * 100, 2)
acc_lgbm

65.58

In [393]:
k_values = list(range(1, 20))
param_grid = {'n_neighbors': k_values}
knn = KNeighborsClassifier()
full_cv_classifier = GridSearchCV(knn, param_grid, cv=5,
                                 scoring='accuracy')

full_cv_classifier.fit(X_train, y_train)

full_cv_classifier.best_estimator_.get_params()
full_pred = full_cv_classifier.predict(X_valid)
knn_acc_gridsearchCV = accuracy_score(full_pred, y_valid)
print('accuracy_score :',knn_acc_gridsearchCV)

accuracy_score : 0.8156424581005587


In [394]:
rfc = RandomForestClassifier(criterion='gini',
                                           n_estimators=1750,
                                           max_depth=7,
                                           min_samples_split=6,
                                           min_samples_leaf=6,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=42,
                                           n_jobs=-1,
                                           verbose=1) 
rfc.fit(X_train,y_train)

preds = rfc.predict(X_valid)

acc_random_forest = round(rfc.score(X_train, y_train) * 100, 2)
acc_random_forest

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1750 out of 1750 | elapsed:    1.3s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 1750 out of 1750 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concur

85.53

In [395]:
X_test

,Pclass,Sex,Fare,Embarked,hasparchild,hassibspouse,Agegroup
PassengerId,,,,,,,
892,0.0,0.0,2.433464,2.0,False,False,2.0
893,0.0,1.0,2.311444,0.0,False,True,3.0
894,1.0,0.0,2.673239,2.0,False,False,5.0
895,0.0,0.0,2.546106,0.0,False,False,1.0
896,0.0,1.0,2.952223,0.0,True,True,1.0
...,...,...,...,...,...,...,...
1305,0.0,0.0,2.464213,0.0,False,False,1.0
1306,2.0,1.0,5.999255,1.0,False,False,2.0
1307,0.0,0.0,2.349387,0.0,False,False,2.0


In [396]:
preds2 = rfc.predict(X_test)
df = pd.DataFrame(columns=['PassengerId','Survived'])
df['PassengerId'] = X_test.index
df['Survived'] = preds2
df[['PassengerId','Survived']].to_csv('submission.csv', index=False)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 1750 out of 1750 | elapsed:    0.1s finished
